In [4]:
from glob import glob
import os
from scipy.io import wavfile
import python_speech_features
import numpy as np
import sounddevice as sd
import time
import joblib
from sklearn.decomposition import PCA
import tensorflow

In [5]:
tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR) # Ignore tensorflow warnings

In [6]:
def read_files(direction, name=''):
    if not os.path.exists(direction):
        raise 'Directory is not available'
    files = '{}/{}*.wav'.format(direction, name)
    return glob(files)

In [7]:
def extract(audio):
    SAMPLE_RATE = 11025
    FRAME_LENGTH = int(SAMPLE_RATE * 0.025)
    FRAME_STEP = FRAME_LENGTH - int(SAMPLE_RATE * 0.015)
    PRE_EMPH = 0.97
    WINDOW_LENGTH = 0.025
    WINDOW_STEP = 0.010
    WINDOW_FUNCTION = np.hamming
    f = []
    
    def calculate_all(feats):
            f.extend(feats.min(axis=0))
            f.extend(feats.max(axis=0))
            f.extend(feats.mean(axis=0))
            f.extend(feats.var(axis=0))
    
    def calculate_energy(frames):
        energies = []
        for frame in frames:
            energy = 1 / len(frame) * np.sum(np.power(frame, 2))
            energies.append(energy)
        return energies
    
    def calculate_zcr(frames):
        def sign(x):
            return 1 if x >= 0 else -1
        
        zc_rates = []
        for frame in frames:
            zc_rate = 0
            for i in range(1, len(frame)):
                zc_rate += abs(sign(frame[i]) - sign(frame[i - 1])) / 2
            zc_rates.append(zc_rate / len(frame))
        return zc_rates
    
    rate, signal = wavfile.read(audio)
    signal_frames = python_speech_features.sigproc.framesig(signal, frame_len=FRAME_LENGTH,
                                                            frame_step=FRAME_STEP,
                                                            winfunc=WINDOW_FUNCTION)
    mfcc = python_speech_features.mfcc(signal_frames, rate, winlen=WINDOW_LENGTH,
                                      winstep=WINDOW_STEP, numcep=13, preemph=PRE_EMPH,
                                      winfunc=WINDOW_FUNCTION)
    delta = np.concatenate([np.zeros(shape=(1, 13)), np.diff(mfcc, n=1, axis=0)])
    delta_delta = np.concatenate([np.zeros(shape=(2, 13)), np.diff(mfcc, n=2, axis=0)])
    
    calculate_all(mfcc)
    calculate_all(delta)
    calculate_all(delta_delta)
    
    energies = calculate_energy(signal_frames)
    calculate_all(np.array([energies]).reshape(len(energies), 1))
    
    zcrs = calculate_zcr(signal_frames)
    calculate_all(np.array([zcrs]).reshape(len(zcrs), 1))
    
    return np.asarray(f)


def batch_extract(direction):
    feats, labels = [], []
    
    valid_samples = read_files(direction, name='ABM')
    for sample in valid_samples:
        sample_feats = extract(sample)
        feats.append(sample_feats)
        labels.append(1)
    
    invalid_samples = read_files(direction, name='NABM')
    for sample in invalid_samples:
        sample_feats = extract(sample)
        feats.append(sample_feats)
        labels.append(0)
    
    return np.asarray(feats), np.asarray(labels)

In [8]:
def record(samplerate=11025, duration=2, play_rec=False, flag='test'):
    FRAMES = int(duration * samplerate)
    print('Recording Started')
    voice = sd.rec(FRAMES, samplerate, dtype=np.float, channels=1, mapping=None, blocking=True)
    print('Recording Ended')
    
    if play_rec is True:
        sd.play(voice, samplerate, blocking=True)
    
    if flag == 'test':
        wavfile.write(filename='temp.wav', rate=samplerate, data=voice)
        time.sleep(1)
    elif flag == 'train':
        return voice, samplerate
    
    
def batch_record(direction, count, name):
    if not os.path.exists(direction):
        os.makedirs(direction)
    
    last_file = len(read_files(direction, name))
    for cnt in range(last_file, last_file + count):
        file_name = '{}/{}.wav'.format(direction, name + str(cnt))
        voice, samplerate = record(flag='train')
        wavfile.write(filename, samplerate, data=voice)
        time.sleep(0.5)

In [9]:
def create_network(feats, labels, n_hiddenlayers=3, _epochs=20, pca_flag=False):
    net = tensorflow.keras.models.Sequential()
    
    net.add(tensorflow.keras.layers.Flatten())
    for _ in range(n_hiddenlayers):
        net.add(tensorflow.keras.layers.Dense(units=128, activation=tensorflow.nn.relu))
    
    net.add(tensorflow.keras.layers.Dense(units=2, activation=tensorflow.nn.softmax))
    net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    if pca_flag is True:
        pca = joblib.load('pca_dnn.pkl')
        feats = pca.transform(feats)
    
    net.fit(feats, labels, epochs=_epochs)
    net.save('net.h5')
    
    
def predict(net, audio, pca_flag=False):
    feats = extract(audio)
    
    if pca_flag is True:
        pca = joblib.load('pca_dnn.pkl')
        feats = pca.transform(feats.reshape(1, -1))
        
    score = net.predict(feats.reshape(1, -1))
    res_class = net.predict_classes(feats.reshape(1, -1))
    
    if res_class == 1:
        print('\n\nAli Bayat Mokhtari Verified.')
    elif res_class == 0:
        print('\n\nNot Verified!')
        
    print('score is: {}'.format(score))
    return res_class


def single_test():
    record(play_rec=True)
    net = tensorflow.keras.models.load_model('net.h5')
    res = predict(net, 'temp.wav', pca_flag=True)
    print(res)
    return res


def calculate_accuracy(direction, pcaflag, name, cls):
    net = tensorflow.keras.models.load_model('net.h5')
    files = read_files(direction, name=name)
    cnt = 0
    for _file in files:
        test_class = predict(net, audio=_file, pca_flag=pcaflag)
        print(_file, test_class)
        if test_class == cls:
            cnt += 1
    print('\n\naccuracy is: {}'.format((cnt / len(files)) * 100))

In [10]:
def create_pca(feats, n_feats):
    pca = PCA(n_components=n_feats, whiten=True)
    pca.fit(feats)
    joblib.dump(pca, filename='pca_dnn.pkl')

In [11]:
feats, labels = batch_extract(direction='./train')
create_pca(feats, n_feats=40)
create_network(feats, labels, n_hiddenlayers=2, _epochs=10, pca_flag=True)

Epoch 1/10
560/560 [==============================] - 0s 217us/sample - loss: 0.5831 - acc: 0.7000
Epoch 2/10
560/560 [==============================] - 0s 71us/sample - loss: 0.3374 - acc: 0.9250
Epoch 3/10
560/560 [==============================] - 0s 72us/sample - loss: 0.1844 - acc: 0.9536
Epoch 4/10
560/560 [==============================] - 0s 94us/sample - loss: 0.1088 - acc: 0.9750
Epoch 5/10
560/560 [==============================] - 0s 79us/sample - loss: 0.0701 - acc: 0.9875
Epoch 6/10
560/560 [==============================] - 0s 80us/sample - loss: 0.0471 - acc: 0.9946
Epoch 7/10
560/560 [==============================] - 0s 81us/sample - loss: 0.0308 - acc: 1.0000
Epoch 8/10
560/560 [==============================] - 0s 60us/sample - loss: 0.0219 - acc: 1.0000
Epoch 9/10
560/560 [==============================] - 0s 83us/sample - loss: 0.0156 - acc: 1.0000
Epoch 10/10
560/560 [==============================] - 0s 75us/sample - loss: 0.0115 - acc: 1.0000


In [12]:
ans = single_test()

Recording Started
Recording Ended


Ali Bayat Mokhtari Verified.
score is: [[0.01602994 0.98397   ]]
[1]


In [ ]:
calculate_accuracy(direction='./train', pcaflag=True, name='NABM', cls=0)